In [12]:
# Implement according to: https://github.com/bobbens/sketch_simplification
# Paper: https://arxiv.org/pdf/1703.08966.pdf

# pytorch 0.4.1

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torchvision.utils import save_image
from torch.utils.serialization import load_lua
from PIL import Image

conv_ske_dic = 'E:/Code/Anime Faces/out/'
simp_ske_dic = './Data/input_ps/'

cache  = load_lua('./Models/simplify _sketch_model.t7', long_size = 8)
model  = cache.model
immean = cache.mean
imstd  = cache.std
model.evaluate()
use_cuda = torch.cuda.device_count() > 0

<bound method Container.parameters of nn.Sequential {
  [input -> (0) -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> (21) -> (22) -> (23) -> (24) -> (25) -> (26) -> (27) -> (28) -> (29) -> (30) -> (31) -> (32) -> (33) -> (34) -> (35) -> (36) -> (37) -> (38) -> (39) -> (40) -> (41) -> (42) -> (43) -> (44) -> (45) -> output]
  (0): nn.SpatialConvolution(1 -> 48, 5x5, 2, 2, 2, 2)
  (1): nn.ReLU
  (2): nn.SpatialConvolution(48 -> 128, 3x3, 1, 1, 1, 1)
  (3): nn.ReLU
  (4): nn.SpatialConvolution(128 -> 128, 3x3, 1, 1, 1, 1)
  (5): nn.ReLU
  (6): nn.SpatialConvolution(128 -> 128, 3x3, 2, 2, 1, 1)
  (7): nn.ReLU
  (8): nn.SpatialConvolution(128 -> 256, 3x3, 1, 1, 1, 1)
  (9): nn.ReLU
  (10): nn.SpatialConvolution(256 -> 256, 3x3, 1, 1, 1, 1)
  (11): nn.ReLU
  (12): nn.SpatialConvolution(256 -> 256, 3x3, 2, 2, 1, 1)
  (13): nn.ReLU
  (14): nn.SpatialConvolution(256 -> 512, 3x3, 1, 1, 1, 1

In [2]:
# names = [f for f in os.listdir(conv_ske_dic) if f.endswith('.png')]
# name_file = open('name_file.txt', 'a')
# for n in names:
#     if n == names[-1]:
#         name_file.write(n)
#     else:
#         name_file.write(n + ',')
# name_file.close()

In [3]:
class AnimeSketchDataset(Dataset):
    """Anime Sketches dataset converted from the original pictures"""
    
    def __init__(self, img_names, root_dir, transform=None):
        
        """
        Args:
            img_names (string): a path to a txt file of sketches file names.
            root_dir (string): directory with all converted sketches.
            transform (callable, optional): optional transform to be applied on a sample.
        """
        
        self.names = open(img_names, 'r').read().split(',')
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        
        img = os.path.join(self.root_dir, self.names[idx])
        data = Image.open(img).convert('L')
        
        if self.transform:
            data = self.transform(data)
            
        return {'image': data, 'image_name': self.names[idx]}
        
class Preprocess():
    """Preprocess the images as proposed in the paper"""
    
    def __init__(self, immean, imstd):
        
        self.mean = immean
        self.std = imstd
    
    def __call__(self, data):
        
        w, h = data.size[0], data.size[1]
        pw = 8 - (w%8) if w%8 != 0 else 0
        ph = 8 - (h%8) if h%8 != 0 else 0
        
        data = (transforms.ToTensor()(data) - immean) / imstd
        
        if pw!=0 or ph!=0:
            data = torch.nn.ReplicationPad2d((0, pw, 0, ph))(data).data
            
        return data

In [9]:
dataset = AnimeSketchDataset('name_file.txt', conv_ske_dic, Preprocess(immean, imstd))
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

In [17]:
for inputs in dataloader:
    data, img = inputs['image'], inputs['image_name']
    if use_cuda:
        pred = model.cuda().forward(data.cuda()).float()
    else:
        pred = model.toforward(data)
    for i in range(data.shape[0]):
        save_image(pred[i], simp_ske_dic + img[i])

AttributeError: 'dict' object has no attribute 'cuda'